In [4]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split

In [5]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_4316\3030752658.py:1: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("./data/train.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,20025,833,642,51,2,1,1,1,1,9.0,...,3.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,922731975
1,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923172525
2,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923833350
3,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,925195050
4,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,926576775


In [6]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_4316\2707460454.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("./data/test.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,driverStandingsId,raceId_y,points_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,23949,997,4,1,14,13,8,8,8,4.0,...,12474,93,16.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,298739826
1,23949,997,4,1,14,13,8,8,8,4.0,...,12494,94,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299218806
2,23949,997,4,1,14,13,8,8,8,4.0,...,12514,95,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299697786
3,23949,997,4,1,14,13,8,8,8,4.0,...,12534,96,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300176766
4,23949,997,4,1,14,13,8,8,8,4.0,...,12554,97,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300655746


In [7]:
validation_df = pd.read_csv("./data/validation.csv")
validation_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_4316\3375616066.py:1: DtypeWarning: Columns (13,16,36) have mixed types. Specify dtype option on import or set low_memory=False.
  validation_df = pd.read_csv("./data/validation.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,22109,899,17,9,2,4,2,2,2,18.0,...,28.0,8,8,0,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1410907944
1,22109,899,17,9,2,4,2,2,2,18.0,...,53.0,4,4,1,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1411438560
2,22109,899,17,9,2,4,2,2,2,18.0,...,78.0,1,1,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1412499792
3,22109,899,17,9,2,4,2,2,2,18.0,...,93.0,1,1,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1413030408
4,22109,899,17,9,2,4,2,2,2,18.0,...,103.0,3,3,2,red_bull,Red Bull,Austrian,http://en.wikipedia.org/wiki/Red_Bull_Racing,Finished,1413561024


In [80]:
new_train_df = train_df[
    ["result_driver_standing", "points_y", "company", "round",
     "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId", "position"]]
new_train_df.head()

,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId,position
0,922731975,3.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,3
1,923172525,6.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,3
2,923833350,6.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,4
3,925195050,10.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,3
4,926576775,10.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,4


In [81]:
new_validation_df = validation_df[
    ["result_driver_standing", "points_y", "company", "round",
     "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId", "position"]]
new_validation_df.head()

,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId,position
0,1410907944,28.0,Red Bull,19,0,Brazilian Grand Prix,2013,18.0,2,22109,899,8
1,1411438560,53.0,Red Bull,19,1,Brazilian Grand Prix,2013,18.0,2,22109,899,4
2,1412499792,78.0,Red Bull,19,2,Brazilian Grand Prix,2013,18.0,2,22109,899,1
3,1413030408,93.0,Red Bull,19,2,Brazilian Grand Prix,2013,18.0,2,22109,899,1
4,1413561024,103.0,Red Bull,19,2,Brazilian Grand Prix,2013,18.0,2,22109,899,3


In [82]:
new_test_df = test_df[
    ["result_driver_standing", "points_y", "company", "round",
    "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId"]]
new_test_df

,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId
0,298739826,16.0,McLaren,9,0,Austrian Grand Prix,2018,4.0,8,23949,997
1,299218806,21.0,McLaren,9,0,Austrian Grand Prix,2018,4.0,8,23949,997
2,299697786,21.0,McLaren,9,0,Austrian Grand Prix,2018,4.0,8,23949,997
3,300176766,25.0,McLaren,9,0,Austrian Grand Prix,2018,4.0,8,23949,997
4,300655746,25.0,McLaren,9,0,Austrian Grand Prix,2018,4.0,8,23949,997
...,...,...,...,...,...,...,...,...,...,...,...
352923,1880337225,5.0,McLaren,12,0,Belgian Grand Prix,2023,0.0,20,26085,1110
352924,1881380625,5.0,McLaren,12,0,Belgian Grand Prix,2023,0.0,20,26085,1110
352925,1881902325,17.0,McLaren,12,0,Belgian Grand Prix,2023,0.0,20,26085,1110
352926,1882424025,27.0,McLaren,12,0,Belgian Grand Prix,2023,0.0,20,26085,1110


In [83]:
# finished_train = new_train_df[new_train_df["status"] == "Finished"]

finished_train = new_train_df
# finished_train.loc[finished_train['status'] != 'Finished', 'status'] = 'Unfinished'

In [84]:
# finished_test = new_test_df[new_test_df["status"] == "Finished"]
finished_test = new_test_df
# finished_test.loc[finished_test['status'] != 'Finished', 'status'] = 'Unfinished'

In [85]:
# finished_validation = new_validation_df[new_validation_df["status"] == "Finished"]
finished_validation = new_validation_df
# finished_validation.loc[finished_validation['status'] != 'Finished', 'status'] = 'Unfinished'

In [86]:
finished_train = finished_train.fillna(0)
finished_test = finished_test.fillna(0)
finished_validation = finished_validation.fillna(0)

In [95]:
# encoding all columns
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_train[column] = le.fit_transform(finished_train[column])
    label_encoders[column] = le
    
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_test[column] = le.fit_transform(finished_test[column])
    label_encoders[column] = le

label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_validation[column] = le.fit_transform(finished_validation[column])
    label_encoders[column] = le


In [96]:
params = {
    'objective': 'regression',
    'metric': 'rmse',  # RMSE for regression
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'device': 'gpu',  # Use GPU for training
    'gpu_platform_id': 0,  # Specify GPU platform ID (0 by default)
    'gpu_device_id': 0  # Specify GPU device ID (0 by default)
}

In [98]:
X_train = finished_train.drop(columns=['position'])
y_train = finished_train['position']

X_test = finished_validation.drop(columns=['position'])
y_test = finished_validation['position']

predictkardo = finished_test
predictkardo
# y_test = finished_test['position']


,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId
0,298739826,16.0,7,9,0,3,0,4.0,8,23949,997
1,299218806,21.0,7,9,0,3,0,4.0,8,23949,997
2,299697786,21.0,7,9,0,3,0,4.0,8,23949,997
3,300176766,25.0,7,9,0,3,0,4.0,8,23949,997
4,300655746,25.0,7,9,0,3,0,4.0,8,23949,997
...,...,...,...,...,...,...,...,...,...,...,...
352923,1880337225,5.0,7,12,0,6,5,0.0,20,26085,1110
352924,1881380625,5.0,7,12,0,6,5,0.0,20,26085,1110
352925,1881902325,17.0,7,12,0,6,5,0.0,20,26085,1110
352926,1882424025,27.0,7,12,0,6,5,0.0,20,26085,1110


In [99]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Train the model
num_round = 5000
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1389
[LightGBM] [Info] Number of data points in the train set: 2830101, number of used features: 11
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (32.39 MB) transferred to GPU in 0.040663 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 12.332001
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (25.92 MB) transferred to GPU in 0.022561 secs. 1 sparse feature groups
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 9 dense feature groups (25.91 MB) transferred to GPU in 0.021202 secs. 1 sparse feature groups
[LightGBM] [Info] Size of histogram bi

In [100]:

# Predict on the test set
# y_pred = bst.predict(predictkardo, num_iteration=bst.best_iteration)
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

c:\Users\krish\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [101]:
print(f'RMSE: {rmse}')
print(f'R²: {r2}')

# Optionally, save the model
# bst.save_model('lightgbm_model.txt')

# Print sample predictions
predictions_df = X_test.copy()
predictions_df['actual_position'] = y_test
predictions_df['position'] = y_pred
predictions_df
# submissions = predictions_df[['position', 'result_driver_standing']]
# print(submissions)

RMSE: 8.107899124960188
R²: -0.8746336305552078


,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId,actual_position,position
0,1410907944,28.0,9,19,0,6,0,18.0,2,22109,899,8,8.374899
1,1411438560,53.0,9,19,1,6,0,18.0,2,22109,899,4,6.414342
2,1412499792,78.0,9,19,2,6,0,18.0,2,22109,899,1,6.756935
3,1413030408,93.0,9,19,2,6,0,18.0,2,22109,899,1,5.386139
4,1413561024,103.0,9,19,2,6,0,18.0,2,22109,899,3,6.661353
...,...,...,...,...,...,...,...,...,...,...,...,...,...
353757,1705168800,123.0,7,9,6,2,5,4.0,8,23949,997,1,6.154174
353758,1705815423,133.0,7,9,7,2,5,4.0,8,23949,997,1,6.027173
353759,297542376,6.0,7,9,0,2,5,4.0,8,23949,997,3,14.870200
353760,297877662,8.0,7,9,0,2,5,4.0,8,23949,997,5,14.357299


In [29]:
# save submission to csv
submissions.to_csv('submission.csv', index=False)